In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import random
from PIL import Image
import cv2

from tqdm import tqdm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
path = '/home/jovyan/hfactory_magic_folders/tooling_for_the_data_scientist/deepfakes_detection/'

In [ ]:
test = pd.read_csv(path + 'test.csv')

In [ ]:
train = pd.read_csv(path + 'train.csv')

In [ ]:
sample_submission = pd.read_csv(path + 'sample_submission.csv', sep=';')

In [ ]:
train[train['image_id'] == '00059efd-9bd6-4493-9b0b-c6d7fafa4be2']

In [ ]:
'00059efd-9bd6-4493-9b0b-c6d7fafa4be2'

In [ ]:
'0007258f-1fa9-4b40-b663-f0eeed90acc5'

In [ ]:
train[train['image_id'] == '00059efd-9bd6-4493-9b0b-c6d7fafa4be2']

In [ ]:
train.shape[0] + test.shape[0]

In [ ]:
cv2.imread("hfactory_magic_folders/tooling_for_the_data_scientist/deepfakes_detection/images/"+train.loc[0,'image_id']+".jpg")

In [ ]:
plt.figure(figsize=(15,15))
for cur,i in enumerate(train.index[0:25]):
    plt.subplot(5,5,cur+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    
    plt.imshow(cv2.imread(path + "/images/"+train.loc[i,'image_id']+".jpg"))
    
    plt.xlabel(train.loc[i,"label"])
        
plt.show()

In [ ]:
train[train['label'] == 0].image_id

In [ ]:
image_size = (256, 256)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "train_images",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "train_images",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size
)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")


In [ ]:
input_shape = (None, 256, 256)

inputs = keras.Input(shape=input_shape)
# x = data_augmentation(inputs)
x = layers.experimental.preprocessing.Rescaling(scale=1./255)(inputs)
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

In [ ]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    # x = data_augmentation(inputs)

    # Entry block
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)

In [ ]:
model = make_model(input_shape=image_size + (3,), num_classes=2)
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
epochs = 2

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds.take(10), epochs=epochs, callbacks=callbacks, validation_data=val_ds.take(3),
)
